# POKÉMON BATTLE SIMULATOR TESTING

In [ ]:
class Movement:
    """Represents a Pokémon movement"""
    def __init__(self, name, ptype, damage, accuracy=100):
        """
        Args:
            name = Name of the movement
            ptype = Movement type
            damage = Power of the movement
            accuracy = Precission (0-100)
        """
        self.name = name
        self.ptype = ptype
        self.damage = damage
        self.accuracy = accuracy

    def __str__(self):
        return f"{self.name} ({self.ptype} - Power {self.damage}, Accuracy {self.accuracy})"

In [ ]:
class Movement:
    def __init__(self, name, power, ptype, accuracy=100):
        self.name = name
        self.power = power
        self.ptype = ptype
        self.accuracy = accuracy
    
    def __str__(self):
        return f"{self.name} ({self.ptype}) - Power: {self.power}, Accuracy: {self.accuracy}%"
    
    def __repr__(self):
        return f"Movement('{self.name}', {self.power}, '{self.ptype}', {self.accuracy})"



class Pokemon:
    def __init__(self, name, max_hp, ptype=None, movements=None):
        self.name = name
        self._max_hp = max_hp
        self.__actual_hp = max_hp
        
        # IMPORTANT: Assign FIRST to private variables
        self._ptype = ptype if ptype is not None else []
        self._movements = movements if movements is not None else []
        
        # THEN validate and limit
        if len(self._ptype) > 2:
            self._ptype = self._ptype[:2]
        
        if len(self._movements) > 4:
            self._movements = self._movements[:4]
    
    # ============== HP PROPERTY ==============
    @property
    def hp(self):
        """Returns actual hp"""
        return self.__actual_hp
    
    @hp.setter
    def hp(self, value):
        """Allows for the secure modification of the Pokemon's HP"""
        if value < 0:
            self.__actual_hp = 0
        elif value > self._max_hp:
            self.__actual_hp = self._max_hp
        else:
            self.__actual_hp = value
    
    # ============== PTYPE PROPERTY ==============
    @property
    def ptype(self):
        """Returns the Pokémon's types"""
        return self._ptype  # ✅ With underscore
    
    @ptype.setter
    def ptype(self, new_ptype):
        """
        Setter: Validates that the Pokémon doesn't have more than 2 types
        If you try to give it more than 2, it will only save the first 2.
        """
        if not isinstance(new_ptype, list):
            raise ValueError("ptype must be a list")
        
        self._ptype = new_ptype[:2]  # ✅ With underscore
    
    # ============== MOVEMENTS PROPERTY ==============
    @property
    def movements(self):
        """Returns the complete list of movements"""
        return self._movements  # ✅ With underscore - THIS WAS THE ERROR
    
    @movements.setter
    def movements(self, new_movements):
        """
        Setter: Validates that the Pokémon doesn't have more than 4 moves
        If you try to give it more than 4, it will only save the first 4 moves.
        """
        if not isinstance(new_movements, list):
            raise ValueError("movements must be a list")
        
        # Validate all moves are Movement objects
        for move in new_movements:
            if not isinstance(move, Movement):
                raise ValueError("All items must be Movement objects")
        
        # Limit to 4 movements
        self._movements = new_movements[:4]  # ✅ With underscore
    
    # ==================== INTERACTIVE LEARNING SYSTEM ====================
    def learn_movement(self, new_movement):
        """
        Interactive system to learn a new movement.
        If the Pokemon already knows 4 moves, prompts user to choose which to forget.
        
        Args:
            new_movement (Movement): The movement to learn
        
        Returns:
            dict: Result of the operation
        """
        # Validate it's a Movement object
        if not isinstance(new_movement, Movement):
            return {
                "success": False,
                "message": "Must provide a Movement object"
            }
        
        # Check if already knows this movement
        if new_movement in self._movements:
            return {
                "success": False,
                "message": f"{self.name} already knows {new_movement.name}!"
            }
        
        # CASE 1: Has space (less than 4 movements)
        if len(self._movements) < 4:  # ✅ LESS THAN, not greater than
            self._movements.append(new_movement)
            return {
                "success": True,
                "message": f"{self.name} learned {new_movement.name}!"
            }
        
        # CASE 2: Full (4 movements) - Interactive choice
        print(f"\n{'='*60}")
        print(f"{self.name} wants to learn {new_movement.name}!")
        print(f"But {self.name} already knows 4 moves.")
        print(f"{'='*60}\n")
        
        print("Current moves:")
        for i, move in enumerate(self._movements):
            print(f"  {i+1}. {move.name} - Power: {move.power}, Accuracy: {move.accuracy}%")
        
        print(f"\nNew move:")
        print(f"  • {new_movement.name} - Power: {new_movement.power}, Accuracy: {new_movement.accuracy}%")
        
        # Ask which move to forget
        while True:
            print(f"\n{'─'*60}")
            forget_choice = input(f"Which move should {self.name} forget? (enter move name or number 1-4): ").strip()
            
            # Try to match by name (case insensitive)
            move_to_forget = None
            forget_index = None
            
            # Check if input is a number
            if forget_choice.isdigit():
                index = int(forget_choice) - 1
                if 0 <= index < 4:
                    move_to_forget = self._movements[index]
                    forget_index = index
            else:
                # Check if input matches a move name (case insensitive)
                for i, move in enumerate(self._movements):
                    if move.name.lower() == forget_choice.lower():
                        move_to_forget = move
                        forget_index = i
                        break
            
            if move_to_forget is None:
                print(f"❌ '{forget_choice}' is not valid. Please try again.")
                continue
            
            # Confirm the choice
            print(f"\n{self.name} will forget {move_to_forget.name} to learn {new_movement.name}.")
            confirm = input("Are you sure? (yes/no): ").strip().lower()
            
            if confirm in ['yes', 'y', 'yeah', 'yep', 'sure', 'A']:
                # Replace the movement
                self._movements[forget_index] = new_movement
                print(f"\n✅ {self.name} forgot {move_to_forget.name}!")
                print(f"✅ {self.name} learned {new_movement.name}!")
                
                return {
                    "success": True,
                    "message": f"{self.name} learned {new_movement.name}!",
                    "forgotten": move_to_forget,
                    "learned": new_movement
                }
            elif confirm in ['no', 'n', 'nope', 'nah', 'B']:
                print(f"\n{self.name} did not learn {new_movement.name}.")
                return {
                    "success": False,
                    "message": f"{self.name} did not learn {new_movement.name}."
                }
            else:
                print("❌ Please answer 'yes' or 'no'.")
    
    def show_info(self):
        """Display Pokemon information"""
        types_str = "/".join(self._ptype) if self._ptype else "No type"
        print(f"\n{'='*60}")
        print(f"📛 Name: {self.name}")
        print(f"🏷️  Type: {types_str}")
        print(f"❤️  HP: {self.hp}/{self._max_hp}")
        print(f"⚔️  Moves ({len(self._movements)}/4):")
        if self._movements:
            for i, move in enumerate(self._movements):
                print(f"   {i+1}. {move}")
        else:
            print("   (No moves learned yet)")
        print(f"{'='*60}\n")
    
    def is_alive(self):
        """Check if Pokemon is alive"""
        return self.hp > 0
    
    def __str__(self):
        status = "💚" if self.is_alive() else "💀"
        types_str = "/".join(self._ptype) if self._ptype else "???"
        return f"{status} {self.name} ({types_str}) - HP: {self.hp}/{self._max_hp} | Moves: {len(self._movements)}/4"
    
    def __repr__(self):
        return f"Pokemon('{self.name}', {self._max_hp}, {self._ptype})"


In [12]:
thunderbolt = Movement("Thunderbolt", 90, "Electric", 100)
quick_attack = Movement("Quick Attack", 40, "Normal", 100)
iron_tail = Movement("Iron Tail", 100, "Steel", 75)
electro_ball = Movement("Electro Ball", 80, "Electric", 100)
thunder = Movement("Thunder", 110, "Electric", 70)
tackle = Movement("Tackle", 35, "Normal", 100)

In [17]:
pikachu = Pokemon("Pikachu", 35, ["Electric"], [thunderbolt, quick_attack, iron_tail, electro_ball])
pikachu.show_info()
pikachu.hp


📛 Name: Pikachu
🏷️  Type: Electric
❤️  HP: 35/35
⚔️  Moves (4/4):
   1. Thunderbolt (Electric) - Power: 90, Accuracy: 100%
   2. Quick Attack (Normal) - Power: 40, Accuracy: 100%
   3. Iron Tail (Steel) - Power: 100, Accuracy: 75%
   4. Electro Ball (Electric) - Power: 80, Accuracy: 100%



35

In [28]:
pikachu.movements

[Movement('Thunderbolt', 90, 'Electric', 100),
 Movement('Quick Attack', 40, 'Normal', 100),
 Movement('Thunder', 110, 'Electric', 70),
 Movement('Electro Ball', 80, 'Electric', 100)]

In [25]:
pikachu.__str__()

'💚 Pikachu (Electric) - HP: 35/35 | Moves: 4/4'

In [23]:
pikachu.movements
pikachu.is_alive()

True